Can the use of emoji's soften the sentiment of a comment

https://datascienceplus.com/machine-learning-with-text-in-pyspark-part-1/

### Preprocessing

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import Bucketizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Correlation

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# read text file into pyspark dataframe
file = '/project/ds5559/r-slash-group8/sample.csv'
df = spark.read.csv(file,  inferSchema=True, header = True)

In [4]:
df.show(5)
df.count() #15317725 rows

+--------------------------------+-----------+----+------------+---------+----------+------------+----------------------+-----------------+---------+-------+--------------+------+-----+--------+------------+-----+------------+--------------------+-------------+------+----------------+----------+
|                             _c0|created_utc| ups|subreddit_id|  link_id|      name|score_hidden|author_flair_css_class|author_flair_text|subreddit|     id|removal_reason|gilded|downs|archived|      author|score|retrieved_on|                body|distinguished|edited|controversiality| parent_id|
+--------------------------------+-----------+----+------------+---------+----------+------------+----------------------+-----------------+---------+-------+--------------+------+-----+--------+------------+-----+------------+--------------------+-------------+------+----------------+----------+
|                               1| 1430438400|   4|    t5_378oi|t3_34di91|t1_cqug90g|           0|           

15317725

In [5]:
df2 = df.filter(df.author_flair_text != "NA")
df2 = df2.filter(df.author_flair_text != "-") 
df2 = df2.filter(df.author_flair_text != " - ")
df2 = df2.filter(df.author_flair_text.isNotNull())

In [6]:
df2.show(5)
df2.count() #2724362 rows

+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+
|_c0|created_utc|ups|subreddit_id|  link_id|      name|score_hidden|author_flair_css_class|   author_flair_text|      subreddit|     id|removal_reason|gilded|downs|archived|   author|score|retrieved_on|                body|distinguished|edited|controversiality| parent_id|
+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+
|  2| 1430438400|  4|    t5_2qo4s|t3_34g8mx|t1_cqug90h|           0|                  Heat|                Heat|            nba|cqug90h|            NA|     0|    0|       0|WyaOfWad

2724362

In [7]:
df3 = df2.filter(df.author_flair_css_class != "NA")
df3 = df3.filter(df.author_flair_css_class != "-")
df2 = df2.filter(df.author_flair_css_class != " - ")
df3 = df3.filter(df.author_flair_css_class.isNotNull())

In [8]:
df3.show(5)
df3.count() #2117131 rows

+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+
|_c0|created_utc|ups|subreddit_id|  link_id|      name|score_hidden|author_flair_css_class|   author_flair_text|      subreddit|     id|removal_reason|gilded|downs|archived|   author|score|retrieved_on|                body|distinguished|edited|controversiality| parent_id|
+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+
|  2| 1430438400|  4|    t5_2qo4s|t3_34g8mx|t1_cqug90h|           0|                  Heat|                Heat|            nba|cqug90h|            NA|     0|    0|       0|WyaOfWad

2117131

In [9]:
from pyspark.sql import functions as f
df4 = df3.withColumn('label', f.when(f.col('removal_reason') == "NA", 0.0).otherwise(1.0))

In [10]:
df4.show(5)
df4.count() #2117131 rows

+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+-----+
|_c0|created_utc|ups|subreddit_id|  link_id|      name|score_hidden|author_flair_css_class|   author_flair_text|      subreddit|     id|removal_reason|gilded|downs|archived|   author|score|retrieved_on|                body|distinguished|edited|controversiality| parent_id|label|
+---+-----------+---+------------+---------+----------+------------+----------------------+--------------------+---------------+-------+--------------+------+-----+--------+---------+-----+------------+--------------------+-------------+------+----------------+----------+-----+
|  2| 1430438400|  4|    t5_2qo4s|t3_34g8mx|t1_cqug90h|           0|                  Heat|                Heat|            nba|cqug90h|            NA|     0|    0

2117131

In [11]:
df4.groupBy("label").count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  0.0|2002268|
|  1.0| 114863|
+-----+-------+



In [12]:
df4.groupBy("author_flair_css_class", "label").count().show()

+----------------------+-----+-----+
|author_flair_css_class|label|count|
+----------------------+-----+-----+
|                texans|  0.0| 2737|
|                   nym|  0.0|  556|
|                chc-18|  0.0|  478|
|              Celtics2|  0.0| 2477|
|               cheater|  0.0| 1491|
|              Guardian|  0.0|  446|
|                 pedro|  0.0|   40|
|                    bb|  1.0|    1|
|   they are either ...|  1.0|    1|
|              IowaOaks|  0.0|    9|
|             linkseven|  0.0|    2|
|              beyounce|  0.0|   37|
|                 limbo|  0.0|  305|
|   which kept me dr...|  1.0|    1|
|                00FF00|  0.0|  102|
|                  Fett|  0.0|   76|
|                   149|  0.0|  162|
|                MCC-48|  0.0|    6|
|   which assigns a ...|  1.0|    1|
|                  read|  0.0|   36|
+----------------------+-----+-----+
only showing top 20 rows



In [13]:
#select the columns we want to use
df5 = df4.select("author_flair_css_class", "label")
df5.show()

+----------------------+-----+
|author_flair_css_class|label|
+----------------------+-----+
|                  Heat|  0.0|
|                T10B10|  0.0|
|                fan vp|  0.0|
|             FZeroLogo|  0.0|
|      brisbane-broncos|  0.0|
|                 week4|  0.0|
|               press-6|  0.0|
|                   etc|  1.0|
|                Spurs2|  0.0|
|                    is|  0.0|
|  pansear charmeleo...|  0.0|
|              Raptors5|  0.0|
|           LeagueFlair|  0.0|
|                saints|  0.0|
|              steelers|  0.0|
|                     0|  1.0|
|     qld-maroons-retro|  0.0|
|                purple|  0.0|
|                  GIRL|  0.0|
|           fortyniners|  0.0|
+----------------------+-----+
only showing top 20 rows



### Tokenize flairs

In [14]:
from pyspark.ml.feature import  Tokenizer

In [15]:
tokenizer = Tokenizer(inputCol="author_flair_css_class", outputCol="flairClass")
df6 = tokenizer.transform(df5)

In [16]:
df6.show()

+----------------------+-----+--------------------+
|author_flair_css_class|label|          flairClass|
+----------------------+-----+--------------------+
|                  Heat|  0.0|              [heat]|
|                T10B10|  0.0|            [t10b10]|
|                fan vp|  0.0|           [fan, vp]|
|             FZeroLogo|  0.0|         [fzerologo]|
|      brisbane-broncos|  0.0|  [brisbane-broncos]|
|                 week4|  0.0|             [week4]|
|               press-6|  0.0|           [press-6]|
|                   etc|  1.0|             [, etc]|
|                Spurs2|  0.0|            [spurs2]|
|                    is|  0.0|                [is]|
|  pansear charmeleo...|  0.0|[pansear, charmel...|
|              Raptors5|  0.0|          [raptors5]|
|           LeagueFlair|  0.0|       [leagueflair]|
|                saints|  0.0|            [saints]|
|              steelers|  0.0|          [steelers]|
|                     0|  1.0|                 [0]|
|     qld-ma

### Apply CountVectorizer

In [17]:
from pyspark.ml.feature import CountVectorizer

In [18]:
count = CountVectorizer(inputCol="flairClass", outputCol="rawFeatures")
model = count.fit(df6)
df7 = model.transform(df6)
df7.show()

+----------------------+-----+--------------------+--------------------+
|author_flair_css_class|label|          flairClass|         rawFeatures|
+----------------------+-----+--------------------+--------------------+
|                  Heat|  0.0|              [heat]| (77362,[171],[1.0])|
|                T10B10|  0.0|            [t10b10]|(77362,[1695],[1.0])|
|                fan vp|  0.0|           [fan, vp]|(77362,[13,361],[...|
|             FZeroLogo|  0.0|         [fzerologo]|(77362,[8140],[1.0])|
|      brisbane-broncos|  0.0|  [brisbane-broncos]|(77362,[1866],[1.0])|
|                 week4|  0.0|             [week4]|(77362,[1366],[1.0])|
|               press-6|  0.0|           [press-6]|  (77362,[55],[1.0])|
|                   etc|  1.0|             [, etc]|(77362,[1,1275],[...|
|                Spurs2|  0.0|            [spurs2]| (77362,[155],[1.0])|
|                    is|  0.0|                [is]|  (77362,[51],[1.0])|
|  pansear charmeleo...|  0.0|[pansear, charmel...|

In [19]:
# Apply term frequency–inverse document frequency (TF-IDF)
from pyspark.ml.feature import  IDF

In [20]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(df7)
rescaledData = idfModel.transform(df7)
rescaledData.select("label", "features").show() 

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(77362,[171],[6.9...|
|  0.0|(77362,[1695],[9....|
|  0.0|(77362,[13,361],[...|
|  0.0|(77362,[8140],[11...|
|  0.0|(77362,[1866],[9....|
|  0.0|(77362,[1366],[8....|
|  0.0|(77362,[55],[5.98...|
|  1.0|(77362,[1,1275],[...|
|  0.0|(77362,[155],[6.7...|
|  0.0|(77362,[51],[6.09...|
|  0.0|(77362,[290,2451,...|
|  0.0|(77362,[1096],[8....|
|  0.0|(77362,[123],[6.5...|
|  0.0|(77362,[112],[6.5...|
|  0.0|(77362,[89],[6.35...|
|  1.0|(77362,[0],[3.497...|
|  0.0|(77362,[568],[8.0...|
|  0.0|(77362,[22],[5.25...|
|  0.0|(77362,[172],[6.9...|
|  0.0|(77362,[60],[6.05...|
+-----+--------------------+
only showing top 20 rows



### Split training and test sets

In [21]:
seed = 0
sample, sample2 = rescaledData.randomSplit([0.9,0.1],seed)

In [22]:
trainDF, testDF = sample2.randomSplit([0.8,0.2],seed)

In [23]:
trainDF.count()

169100

In [24]:
testDF.count()

42485

### Logistic Regression

In [51]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import numpy as np
maxIter=10
regParam=0.3
elasticNetParam=0.8

lr = LogisticRegression(featuresCol='features',        
                      labelCol='label',  
                      maxIter=maxIter,
                      regParam=regParam, 
                      elasticNetParam=elasticNetParam)

In [52]:
lrModel = lr.fit(trainDF)

In [54]:
predictions = lrModel.transform(testDF)

In [55]:
evaluator = BinaryClassificationEvaluator()

In [56]:
print("Test Area Under ROC", evaluator.evaluate(predictions))

Test Area Under ROC 0.5


In [57]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(predictions.count())
print("Accuracy : ",accuracy)

Accuracy :  0.9451806519948217
